I scraped ***IGN's Top 100 video games***. First I took the ranking information from the main URL and then scraped all the 100 sub-websites (of games) to get detailed variables

In [1]:
#Importing headers and libraries
import requests
#!pip3 install --user bs4
from bs4 import BeautifulSoup

In [2]:
#Scraping IGN website
def Scrape_N_Parse(url) :
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}
    response = requests.get(url, headers = headers)
    parsed_response = BeautifulSoup(response.content, "html.parser")
    return parsed_response

In [3]:
#Parsing the IGN site
IGN_response = Scrape_N_Parse('https://www.ign.com/lists/top-100-games/100/')

In [4]:
#Collecting Ranking information
game_name = IGN_response.find_all('div', class_='item-heading')
game_rank = IGN_response.find_all('div', class_='badge-number')
game_year = IGN_response.find_all('span', class_='item-label-value')
print('Number of Game found : ', len(game_name), 
     ' Number of Rank found : ', len(game_rank),
     ' Number of Year found : ', len(game_year))
game_rank = game_rank[2:] # Removing the contradicting information
print('Number of Game found : ', len(game_name), 
     ' Number of Rank found : ', len(game_rank),
     ' Number of Year found : ', len(game_year))

Number of Game found :  100  Number of Rank found :  102  Number of Year found :  100
Number of Game found :  100  Number of Rank found :  100  Number of Year found :  100


I found two additional ranks. On further checking the html I found that the rank 100 was ***repeated 3 times*** for mentioning the page ranks 100 games at the start, so I ***sliced*** it out

In [5]:
#Collecting Game information
game_link = []
for index in range(100) : 
    game_url = game_name[index].contents[1]['href']
    if game_url == '' : 
        game_url = 'https://www.google.com/search?q=' + game_name[index].text
    game_link.append(game_url)

In [ ]:
#Cleaning Game information
games = []

for index in range(100) :

    Developer = None; Publisher = None; Genre = None; Franchise = None; Platform = None; Release_Date = None;
    
    if 'www.ign.com' in game_link[index] : 
        info = [item.text for item in Scrape_N_Parse(game_link[index]).find_all('div') if "data-cy" in item.attrs]
        for element in info : 
            if 'Developer' in element : 
                Developer = element[element.index(':') + 1 : ].strip(' ')
            elif 'Publisher' in element : 
                Publisher = element[element.index(':') + 1 : ].strip(' ')
            elif 'Genre' in element :  
                Genre = element[element.index(':') + 1 : ].strip(' ')
            elif 'Franchise' in element :  
                Franchise = element[element.index(':') + 1 : ].strip(' ')
            elif 'Platform' in element :  
                Platform = element[element.index(':') + 1 : ].strip(' ')
            elif 'Release Date' in element :  
                Release_Date = element[element.index(':') + 1 : ].strip(' ')
                
    elif 'www.google.com' in game_link[index] : 
        info = Scrape_N_Parse(game_link[index]).find_all('div', class_='rVusze')
        for element in info : 
            if 'Developer' in element.text : 
                Developer = element.text[element.text.index(':') + 1 : ].strip(' ')
            elif 'Publisher' in element.text : 
                Publisher = element.text[element.text.index(':') + 1 : ].strip(' ')
            elif 'Genre' in element.text :  
                Genre = element.text[element.text.index(':') + 1 : ].strip(' ')
            elif 'Series' in element.text :  
                Franchise = element.text[element.text.index(':') + 1 : ].strip(' ')
            elif 'Platform' in element.text :  
                Platform = element.text[element.text.index(':') + 1 : ].strip(' ')
            elif 'Initial release date' in element.text :  
                Release_Date = element.text[element.text.index(':') + 1 : ].strip(' ')
                
    games.append([int(game_rank[index].text), game_name[index].text.strip('\n'), game_year[index].text, Developer, Publisher, Genre, Franchise, Platform, Release_Date])

The above block of code will take ***more time ( atmost 5 mins)***  because of sub-scraping and parsing. I managed to get infomation that is missing from the IGN website from Google

In [ ]:
#Creating a dataframe
import pandas as pd
df = pd.DataFrame(games, columns = ["Rank", "Name", "Year", "Developer", "Publisher", "Genre", "Franchise", "Platform", "Release Date"])
df.set_index("Rank", inplace = True)
df.head(10)

By using the above dataframe can perform basic analysis with respect to the features listed. This can be improved by further scraping to fill up the missing values. The shortcut is to ***using wikipedia*** which has all the infomation hidden or redily available.



In [ ]:
#Incase you are wondering 
df.loc[1:1, :] 

Difficulties Faced: 

***Javascript scraping***: I tried to render the JS first before scraping so that I can avoid the Show Less or Show More events. But there seems to be a specific error with Jupiter Notebook that complicates the process. So I avoided that in this part. 

And, Google does not offer same peice of information for every game, this led to more missing values for older games (for which there was also no infomation in IGN)